In [3]:
%pip install keras-cv
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install tf-models-official
%pip install tensorflow-object-detection-api

     ---------------------------------------- 0.0/84.6 kB ? eta -:--:--
     ------------------ ------------------- 41.0/84.6 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 84.6/84.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/57.4 kB ? eta -:--:--
     ---------------------------------------- 57.4/57.4 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of tf-models-official to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/269.4 kB ? eta -:--:--
     ----------------------------------

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [54 lines of output]
      running egg_info
      writing lib3\PyYAML.egg-info\PKG-INFO
      writing dependency_links to lib3\PyYAML.egg-info\dependency_links.txt
      writing top-level names to lib3\PyYAML.egg-info\top_level.txt
      Traceback (most recent call last):
        File "C:\Users\Nickk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\Nickk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^

     ---------------------------------------- 0.0/577.4 kB ? eta -:--:--
     - ----------------------------------- 30.7/577.4 kB 435.7 kB/s eta 0:00:02
     ---------- --------------------------- 153.6/577.4 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 577.4/577.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Cython-3.0.9-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
Using cached Cython-3.0.9-cp311-cp311-win_amd64.whl (2.8 MB)
   ---------------------------------------- 0.0/54.5 kB ? eta -:--:--
   ---------------------------------------- 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/572.7 kB ? eta -:--:--
   --------------------------------------- 572.7/572.7 kB 35.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/563.7 kB ? eta -:--:--
   --------------------------------------- 563.7/563.7 kB 36.9 MB/s eta 0:00:00
  

In [23]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Split the training data into training and validation sets
val_images = train_images[:5000]
val_labels = train_labels[:5000]
train_images = train_images[5000:]
train_labels = train_labels[5000:]

# Create ImageDataGenerator for resizing
datagen = ImageDataGenerator(
    # Add any other preprocessing here
    rescale=1./255
)

Inference with a pretrained classifier

In [31]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Resizing

# Load MobileNetV2 with ImageNet weights as a base model
# Adjust the input_shape to the new image size
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Create a new model on top
model = Sequential([
    Resizing(32, 32),  # Resizing to 128x128
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare the ImageDataGenerator with the new size
datagen = ImageDataGenerator(
    rescale=1./255
)

# Prepare the data generators
train_gen = datagen.flow(train_images, train_labels, batch_size=32)
val_gen = datagen.flow(val_images, val_labels, batch_size=32)
test_gen = datagen.flow(test_images, test_labels, batch_size=32)

# Train the model
model.fit(train_gen, epochs=2, validation_data=val_gen)

# Evaluate the model
loss, accuracy = model.evaluate(test_gen)
print("Test Accuracy:", accuracy)

Epoch 1/2
1407/1407 [==============================] - 127s 82ms/step - loss: 1.7143 - accuracy: 0.4021 - val_loss: 4.0608 - val_accuracy: 0.1040
Epoch 2/2
313/313 [==============================] - 3s 11ms/step - loss: 4.6435 - accuracy: 0.1001
Test Accuracy: 0.10010000318288803


Fine-tuning a pretrained backbone

In [33]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Split the training data into training and validation sets
val_images = train_images[:5000]
val_labels = train_labels[:5000]
train_images = train_images[5000:]
train_labels = train_labels[5000:]

# Create ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_gen = datagen.flow(train_images, train_labels, batch_size=32)
val_gen = datagen.flow(val_images, val_labels, batch_size=32)
test_gen = datagen.flow(test_images, test_labels, batch_size=32)

# Load MobileNetV2 with ImageNet weights as a base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model
base_model.trainable = False

# Create a new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the top layers
model.fit(train_gen, epochs=2, validation_data=val_gen)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
fine_tune_at = 100  # This is an example, adjust based on model architecture

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

# Compile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model
model.fit(train_gen, epochs=2, validation_data=val_gen)

# Evaluate the model
loss, accuracy = model.evaluate(test_gen)
print("Test Accuracy:", accuracy)

Epoch 1/2
1407/1407 [==============================] - 25s 16ms/step - loss: 2.2817 - accuracy: 0.1289 - val_loss: 2.2518 - val_accuracy: 0.1428
Epoch 2/2
1407/1407 [==============================] - 21s 15ms/step - loss: 2.2255 - accuracy: 0.1574 - val_loss: 2.2036 - val_accuracy: 0.1706
Epoch 1/2
1407/1407 [==============================] - 83s 56ms/step - loss: 3.1038 - accuracy: 0.1548 - val_loss: 3.7479 - val_accuracy: 0.1040
Epoch 2/2
313/313 [==============================] - 4s 11ms/step - loss: 2.3179 - accuracy: 0.0956
Test Accuracy: 0.09560000151395798


Training a image classifier from scratch

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Split the training data into training and validation sets
val_images = train_images[:5000]
val_labels = train_labels[:5000]
train_images = train_images[5000:]
train_labels = train_labels[5000:]

# Create ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Create a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels), batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print("Test Accuracy:", accuracy)

Epoch 1/10
704/704 [==============================] - 8s 10ms/step - loss: 1.3487 - accuracy: 0.5187 - val_loss: 1.1009 - val_accuracy: 0.6032
Epoch 2/10
704/704 [==============================] - 6s 9ms/step - loss: 0.9893 - accuracy: 0.6515 - val_loss: 1.1295 - val_accuracy: 0.6114
Epoch 3/10
704/704 [==============================] - 6s 9ms/step - loss: 0.8382 - accuracy: 0.7081 - val_loss: 0.9759 - val_accuracy: 0.6580
Epoch 4/10
704/704 [==============================] - 6s 9ms/step - loss: 0.7312 - accuracy: 0.7453 - val_loss: 1.0761 - val_accuracy: 0.6400
Epoch 5/10
704/704 [==============================] - 6s 9ms/step - loss: 0.6494 - accuracy: 0.7724 - val_loss: 1.0681 - val_accuracy: 0.6634
Epoch 6/10
704/704 [==============================] - 6s 9ms/step - loss: 0.5750 - accuracy: 0.7977 - val_loss: 1.0591 - val_accuracy: 0.6760
Epoch 7/10
704/704 [==============================] - 6s 9ms/step - loss: 0.5144 - accuracy: 0.8190 - val_loss: 0.9701 - val_accuracy: 0.6948
Epoch